In [1]:
import glob
import geopandas as gpd
import pandas as pd
from datetime import datetime

hr5_cells = gpd.read_file('global-inputs/HR5-cells-beach-slope.gpkg')

def apply_tidal_correction(cell_id):
    slope = hr5_cells.loc[hr5_cells['cell_id'] == cell_id, 'beach_slope'].values[0]
    ref_ele = 0

    folder = f"data/HR5/{cell_id}"
    raw_df = pd.read_csv(f"{folder}/cell_timeseries.csv")
    raw_df['date'] = pd.to_datetime(raw_df['date'])

    tide_df = pd.read_json(f"{folder}/image_metadata.json")
    tide_df['date'] = tide_df['image_date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))

    merged_df = pd.merge(raw_df, tide_df[['date', 'tide_level_msl']], on='date', how='left') # Merge raw_df and tide_df on the date column

    merged_df['tidal_correction'] = (merged_df.tide_level_msl-ref_ele)/slope
    merged_df['corrected_IW_shoreline_position'] = (merged_df.IW_shoreline_chg + merged_df.tidal_correction).round(2)
    merged_df['normalised_IW_shoreline_position'] = (merged_df['corrected_IW_shoreline_position'] - merged_df['corrected_IW_shoreline_position'].iloc[0]).round(2)
    merged_df['normalised_EOV_shoreline_position'] = (merged_df['EOV_shoreline_chg'] - merged_df['EOV_shoreline_chg'].iloc[0]).round(2)

    merged_df.drop_duplicates(subset=['date'], keep='first', inplace=True)
    
    return merged_df

def return_landcover_percentage_change(df):
    landcover = df.drop(columns='date')
    df['total_area'] = landcover.sum(axis=1)
    df['sand_area_percentage'] = (df['sand_area (Ha)']/df['total_area'])*100
    df['water_area_percentage'] = (df['water_area (Ha)']/df['total_area'])*100
    df['vegetation_area_percentage'] = (df['vegetation_area (Ha)']/df['total_area'])*100

    df[['sand_area_percentage_change', 'water_area_percentage_change', 'vegetation_area_percentage_change']] = df[['sand_area_percentage', 'water_area_percentage', 'vegetation_area_percentage']].diff().round(2)

    df.drop_duplicates(subset=['date'], keep='first', inplace=True)
    
    return df[['date', 'sand_area_percentage_change', 'water_area_percentage_change', 'vegetation_area_percentage_change']].fillna(0)

In [2]:
sites_df = pd.read_csv('global-inputs/sites.csv')

cell_ids = list(sites_df['cell_id'])

cells = hr5_cells[hr5_cells['cell_id'].isin(cell_ids)]

test script jupyter notebook 1


In [3]:
cell_ids = cells.cell_id.tolist()
# print(cell_ids)
for cell in cell_ids:
    #print(cell)
    corrected_df = apply_tidal_correction(cell)
    shorelines_df = corrected_df[['date', 'normalised_IW_shoreline_position', 'normalised_EOV_shoreline_position', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg']]
    shorelines_df.rename(columns={'EOV_shoreline_chg': 'EOV_shoreline_position'})
    shorelines_df.to_csv(f"data/HR5/{cell}/shoreline_timeseries_tidal_correction.csv")

    landcover_df = corrected_df[['date', 'sand_area (Ha)', 'water_area (Ha)', 'vegetation_area (Ha)']]
    landcover_df = return_landcover_percentage_change(landcover_df)
    landcover_df.to_csv(f"data/HR5/{cell}/landcover_timeseries_percentage_change.csv")